In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("spam_assassin.csv")
data.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


In [3]:
data.shape

(5796, 2)

In [4]:
data.describe()

,target
count,5796.000000
mean,0.327122
std,0.469203
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [5]:
data.duplicated().sum()

467

In [6]:
data.drop_duplicates(inplace=True)

In [7]:
data.shape

(5329, 2)

In [8]:
data.isnull().sum()

text      0
target    0
dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5329 entries, 0 to 5795
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5329 non-null   object
 1   target  5329 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 124.9+ KB


In [10]:
import nltk 
from nltk.corpus import stopwords
import string 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nyamasi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
def process_text(text):
    #remove puntuations,remove stopwords,  return a list ofclean text
    nopunc =[char for char in text if char  not in string.punctuation]
    nopunc = ''.join(nopunc)
    #2
    clean_words =[word for word  in nopunc.split() if word.lower()not in stopwords.words('english')]
    #3
    return clean_words

In [12]:
#show separting words by commas/tokenazation
data['text'].head().apply(process_text)

0    [ilugadminlinuxie, Mon, Jul, 29, 112802, 2002,...
1    [gort44excitecom, Mon, Jun, 24, 175421, 2002, ...
2    [forkadminxentcom, Mon, Jul, 29, 113957, 2002,...
3    [dcm123btamailnetcn, Mon, Jun, 24, 174923, 200...
4    [ilugadminlinuxie, Mon, Aug, 19, 110247, 2002,...
Name: text, dtype: object

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
messages =CountVectorizer(analyzer=process_text).fit_transform(data['text'])

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(messages,data['target'],test_size=0.2,random_state=42)

In [17]:
messages.shape

(5329, 192304)

In [18]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train,y_train)

In [19]:
#prediction
print(classifier.predict(X_test))
#actual values
print(y_test.values)



[1 1 0 ... 0 0 0]
[1 1 0 ... 0 0 0]


In [22]:
#evaluate the model on testing dataset
from sklearn.metrics import classification_report ,confusion_matrix,accuracy_score
pred =classifier.predict(X_test)
print(classification_report(y_test,pred))
print()
print('confusion matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy :', accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       726
           1       1.00      0.92      0.96       340

    accuracy                           0.97      1066
   macro avg       0.98      0.96      0.97      1066
weighted avg       0.98      0.97      0.97      1066


confusion matrix: 
 [[725   1]
 [ 26 314]]

Accuracy : 0.974671669793621


# Improved model using ensembling

In [25]:
#ensemble modelsss
#bagging 
#decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
scores = cross_val_score(DecisionTreeClassifier(),messages,data['target'],cv=5)
scores

array([0.98311445, 0.98123827, 0.98686679, 0.98217636, 0.98497653])

In [27]:
from sklearn.ensemble import BaggingClassifier

bag_model =BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=100,
max_samples=0.8,
oob_score =True,
random_state=0)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.9821721792165142

In [28]:
bag_model.score(X_test, y_test)

0.9849906191369606

In [ ]:
bag_model =BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=100,
max_samples=0.8,
oob_score =True,
random_state=0)
scores =cross_val_score(bag_model, messages,data['target'], cv=5)
scores.mean()